# 1. Install necessary libraries

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

# 2. Read "category.txt"

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Đặt đường dẫn đến thư mục chứa dữ liệu của bạn


In [5]:
# Đọc danh sách các món ăn từ file 'category.txt'
category_file = '/kaggle/input/food-image-dataset/data/UECFOOD256 2/category.txt'  # Thay 'path_to' bằng đường dẫn thực tế
with open(category_file, 'r') as f:
    categories = [line.strip().split("\t") for line in f.readlines()]

# In ra danh sách các món ăn
categories = categories[1:]
categories[0] = ["1","rice"]
categories[9] = ["10","tempura bowl"]
categories[10] = ["11", "bibimbap"]


# 3. Split images to train and test model

In [6]:
import os
import shutil
import random
from PIL import Image

# Thư mục chứa dữ liệu gốc
DATASET_DIR = "/kaggle/input/food-image-dataset/data/UECFOOD256 2"
IMG_DIR = os.path.join(DATASET_DIR, "images")

# Thư mục mới theo định dạng YOLO
YOLO_DIR = "/kaggle/working/yolo_dataset"
IMG_TRAIN_DIR = os.path.join(YOLO_DIR, "images/train")
IMG_VAL_DIR = os.path.join(YOLO_DIR, "images/val")
LABEL_TRAIN_DIR = os.path.join(YOLO_DIR, "labels/train")
LABEL_VAL_DIR = os.path.join(YOLO_DIR, "labels/val")

# Tạo thư mục YOLO
for d in [IMG_TRAIN_DIR, IMG_VAL_DIR, LABEL_TRAIN_DIR, LABEL_VAL_DIR]:
    os.makedirs(d, exist_ok=True)

# Đọc và chuyển đổi bb_info.txt
for class_id in os.listdir(IMG_DIR):
    class_path = os.path.join(IMG_DIR, class_id)
    bb_info_path = os.path.join(class_path, "bb_info.txt")
    
    if not os.path.isdir(class_path) or not os.path.exists(bb_info_path):
        continue
    
    with open(bb_info_path, "r") as f:
        next(f)
        lines = f.readlines()
    
    image_labels = {}  # Lưu nhãn của từng ảnh

    for line in lines:
        parts = line.strip().split()
        if len(parts) < 5:
            continue
        
        img_name, x1, y1, x2, y2 = parts[0] , int(parts[1]), int(parts[2]), int(parts[3]), int(parts[4])
        img_path = os.path.join(class_path, img_name) + '.jpg'

        
        if not os.path.exists(img_path):
            continue

        # Tính toán bbox theo định dạng YOLO (chuẩn hóa về 0-1)
        original_image = Image.open(img_path)
        original_width, original_height = original_image.size
        x_center = ((x1 + x2) / 2) / original_width
        y_center = ((y1 + y2) / 2) / original_height
        width = (x2 - x1) / original_width
        height = (y2 - y1) / original_height
        
        if width <= 0 or height <= 0 or x_center < 0 or x_center > 1 or y_center < 0 or y_center > 1 or width > 1 or height > 1:
            continue

        
        # Lưu nhãn của ảnh
        if img_name not in image_labels:
            img_name += ".jpg"
            image_labels[img_name] = []
            
        image_labels[img_name].append(f"{int(class_id)-1} {x_center} {y_center} {width} {height}\n")
        

    # Chia tập train/val (80/20)
    img_files = list(image_labels.keys())
    random.shuffle(img_files)
    split_idx = int(0.8 * len(img_files))
    train_files, val_files = img_files[:split_idx], img_files[split_idx:]

    
    for img_name in img_files:
        img_src = os.path.join(class_path, img_name)
        label_content = "".join(image_labels[img_name])
        
        if img_name in train_files:
            img_dst = os.path.join(IMG_TRAIN_DIR, img_name)
            label_dst = os.path.join(LABEL_TRAIN_DIR, img_name.replace(".jpg", ".txt"))
        else:
            img_dst = os.path.join(IMG_VAL_DIR, img_name)
            label_dst = os.path.join(LABEL_VAL_DIR, img_name.replace(".jpg", ".txt"))
        
        # Copy ảnh và ghi file nhãn
        shutil.copy(img_src, img_dst)
        with open(label_dst, "w") as f:
            f.write(label_content)

print("✅ Dữ liệu đã được chuyển đổi thành công!")


✅ Dữ liệu đã được chuyển đổi thành công!


# 4. Check error image and remove it

In [7]:
import os

for label_file in os.listdir(LABEL_VAL_DIR):
    with open(os.path.join(LABEL_VAL_DIR, label_file), "r") as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            if any(float(val) > 1 or float(val) < 0 for val in parts[1:]):
                print(f"⚠️ Bbox lỗi trong file {label_file}: {line}")


# 5. Create file Yaml to train Yolov5 model

In [8]:
# Đường dẫn đến thư mục dữ liệu trên Kaggle
train_path = '/kaggle/working/yolo_dataset/images/train'
val_path = '/kaggle/working/yolo_dataset/images/val'


# Tạo danh sách tên các lớp, trừ đi 1 từ chỉ số lớp
class_names = {int(i[0]) - 1 : i[1] for i in categories} # Đảm bảo các lớp được sắp xếp đúng thứ tự

# Tạo tệp data.yaml
yaml_content = f"""
train: {train_path}
val: {val_path}

nc: {len(class_names)}  
names: {class_names}
"""

# Lưu tệp vào thư mục làm việc
with open('/kaggle/working/data.yaml', 'w') as file:
    file.write(yaml_content)

print("data.yaml đã được tạo thành công!")


data.yaml đã được tạo thành công!


# 6. Train model

In [ ]:
!python train.py --data /kaggle/working/data.yaml --cfg yolov5s.yaml --weights yolov5s.pt --batch-size 16 --epochs 50 --patience 10


# 7. Test model

In [ ]:
!python val.py --data /kaggle/working/data.yaml --weights runs/train/exp/weights/best.pt --batch-size 16
